In [24]:
import pandas as pd
from IPython.display import display

In [25]:
#carrega arquivo que está na pasta:
nome_arquivo = input('digite o nome do arquivo de acordo como salvou na pasta (sem ".xlsx"):' )
file_upload = nome_arquivo+'.xlsx'

In [26]:
#Limpa arquivo deixando apenas as colunas úteis
openpo = pd.read_excel(file_upload,
                      usecols=['BU Country','Location','Location Description','Cost Center Description',
                       'GL Account Description','PO #','PO Line #','PO Creation Date in Source System',
                       'PO Status','PO Line Status','Purpose of Purchase','Commodity Code','EBS PO Category',
                       'Supplier #','Supplier Name','GL Project Description','Need by Date','Ordered Amount (Reporting Currency)',
                       'Remaining Amount (Reporting Currency)']
                       
                                           
                      
                      )

In [27]:
#filtrar apenas PO Abertas
openpo = openpo[openpo['PO Line Status'] == "OPEN"]


In [28]:
#Cria um DF apenas de Opex
openpo_opex = openpo[
                        (openpo['GL Project Description'] == '0000 - DEFAULT PROJECT') |
                        (openpo['GL Project Description'] == '8069 - OPEX ACCELERATION')
]

In [29]:
#Cria um DF apenas de Capex (Projeto)
openpo_capex_projeto = openpo[
                        (openpo['GL Project Description'] == '5300 - INFRA SPOF') |
                        (openpo['GL Project Description'] == '5200 - MAINT-GENERAL')]

In [30]:
#Cria um DF apenas de R&M
openpo_rm = openpo[
                        (openpo['GL Project Description'] == '8001 - NATIONAL MAINT') |
                        (openpo['GL Project Description'] == '8000 - CORRECTIVE MAINTENANCE') |
                        (openpo['GL Project Description'] == '8002 - PREVENTIVE MAINT')

]

In [31]:
# Criar data frame de Opex por país como uma tabela dinâmica
openpo_opex_pais = openpo_opex.pivot_table(index='BU Country',
                                            columns=openpo_opex['Need by Date'].dt.year ,
                                            values='Remaining Amount (Reporting Currency)',
                                            aggfunc='sum'
                                            
                                         )




#penpo_opex_pais'Total Orderd']=openpo_opex_pais[e'otal Ordered'].apply(lambda x: f'$ {x:.1f}'.replace(',','.'))
#openpo_opex_pais['Total Remaining']=openpo_opex_pais['otal Remaining'].apply(lambda x: f'$ {x:.1f}ace(',','.'))

print(openpo_opex_pais.to_string(float_format='$ {:,.1f}'.format))


Need by Date      2023       2024        2025        2026
BU Country                                               
Brazil       $ 1,196.4 $ 39,934.3 $ 927,779.2 $ 348,206.0
Chile              NaN $ 56,173.1 $ 728,380.7         NaN
Colombia           NaN $ 98,100.8 $ 159,338.2         NaN
Mexico       $ 9,389.3  $ 4,109.3 $ 667,007.6         NaN
Peru         $ 1,867.7    $ 137.7  $ 81,700.0         NaN


In [32]:
#Open PO por país e site:
openpo_opex_pais_site = openpo_opex.pivot_table(index=['BU Country','Location Description'],
                                                columns=openpo_opex['Need by Date'].dt.year,
                                                values='Remaining Amount (Reporting Currency)',
                                                aggfunc='sum')

print(openpo_opex_pais_site.to_string(float_format='$ {:,.1f}'.format))


Need by Date                                     2023       2024        2025        2026
BU Country Location Description                                                         
Brazil     0177 - RJ1 RIO DE JANEIRO              NaN  $ 7,748.4  $ 71,610.9   $ 5,067.1
           0178 - RJ2 RIO DE JANEIRO              NaN  $ 2,823.6 $ 141,164.7   $ 2,123.4
           0179 - SP1 SAO PAULO                   NaN        NaN  $ 65,944.1  $ 42,479.5
           0180 - SP2 SAO PAULO                   NaN  $ 8,423.7  $ 71,071.6  $ 42,042.8
           0201 - SP3 SAO PAULO             $ 1,196.4  $ 6,387.6 $ 188,432.7 $ 156,826.1
           0205 - SP4 SAO PAULO                   NaN  $ 8,814.7 $ 256,023.1  $ 94,755.0
           0239 - SAO PAOLO OFFICE SPO            NaN        NaN   $ 1,525.1         NaN
           0240 - RIO DE JANEIRO OFFICE RJO       NaN        NaN     $ 745.3         NaN
           0252 - RJ3 RIO DE JANEIRO              NaN  $ 4,141.3 $ 126,835.1   $ 4,912.2
           0311 - SP5

In [33]:
openpo_country_site_account = openpo_opex.pivot_table(index=['BU Country','Location Description','GL Account Description'],
                                columns=openpo_opex['Need by Date'].dt.year,
                                values='Remaining Amount (Reporting Currency)',
                                aggfunc='sum')

print(openpo_country_site_account.to_string(float_format='$ {:,.1f}'.format))

#with pd.ExcelWriter('OpenPO_Analysis.xlsx') as writer:
   # openpo_opex_pais.to_excel(writer, sheet_name='')
    #openpo_opex_pais_site.to_excel(writer, sheet_name='pais_site')
    #openpo_country_site_account.to_excel(writer, sheet_name='site_conta')


Need by Date                                                                                          2023       2024        2025        2026
BU Country Location Description             GL Account Description                                                                           
Brazil     0177 - RJ1 RIO DE JANEIRO        52502 - EQUIPMENT LEASE                                    NaN        NaN   $ 2,479.2         NaN
                                            52504 - COMPUTER HARDWARE EXPENSE                          NaN  $ 1,863.8  $ 10,976.7         NaN
                                            53501 - JANITORIAL EXPENSE                                 NaN  $ 5,884.7  $ 14,315.1         NaN
                                            53502 - SECURITY                                           NaN        NaN  $ 23,197.4         NaN
                                            53503 - CONSULTING SERVICES                                NaN        NaN   $ 3,823.9   $ 2,123.4
      

In [34]:
#analítico por Site - Fornecedor - PO
open_po_analitico = openpo_opex.pivot_table(index=['BU Country','Location Description','Supplier Name','PO #'],
                                columns=openpo_opex['Need by Date'].dt.year,
                                values='Remaining Amount (Reporting Currency)',
                                aggfunc='sum')

print(open_po_analitico.to_string(float_format='$ {:,.1f}'.format))

with pd.ExcelWriter('OpenPO_Analysis.xlsx') as writer:
    openpo_opex_pais.to_excel(writer, sheet_name='Country')
    openpo_opex_pais_site.to_excel(writer, sheet_name='IBX')
    openpo_country_site_account.to_excel(writer, sheet_name='Account - IBX')
    open_po_analitico.to_excel(writer, sheet_name="Supplier_PO") 
    

Need by Date                                                                                                                          2023       2024        2025       2026
BU Country Location Description             Supplier Name                                                            PO #                                                   
Brazil     0177 - RJ1 RIO DE JANEIRO        DELL COMPUTADORES DO BRASIL LTDA                                         1110412708        NaN     $ 49.1         NaN        NaN
                                                                                                                     1110439526        NaN    $ 118.3         NaN        NaN
                                                                                                                     1110469203        NaN        NaN     $ 160.9        NaN
                                                                                                                     1110502728        